<a href="https://colab.research.google.com/github/usamathakur87/class-projects/blob/main/rag_by_usama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain-pinecone langchain-google-genai

In [2]:
from google.colab import userdata

from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [3]:
import time

index_name = "online-rag-project-4"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

index = pc.Index(index_name)

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ['GOOGLE_API_KEY'] =  userdata.get("GOOGLE_API_KEY")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [5]:
vector = embeddings.embed_query("we are building a RAG Text")

In [6]:
vector[:5]

[0.04060795158147812,
 -0.04385491460561752,
 -0.05683692544698715,
 -0.021194666624069214,
 0.010449625551700592]

In [7]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [8]:
# data save

from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

# data retrive

['5f9bb02c-e550-46ab-b617-62214076a2ee',
 '82a5e304-daf5-4e1d-9944-37b9e0c917f4',
 '34e9000a-24fe-4842-9bc0-d8663c628b85',
 '340df6c1-cad3-4f50-bc3d-21791e265f24',
 '5d1753bc-9df7-437b-88c4-cdc77a96637d',
 '4cd001de-34c1-4ca2-8bd8-f830247c07b0',
 '994b70a6-ab72-462c-9854-0167105e51ad',
 '4a58f277-6f66-4fbf-ad54-642a7c0c2961',
 '6dd9d071-739f-4bed-8507-612f9dd38fae',
 '9ab6ba11-e521-4008-862d-a0f99c907f4b']

In [9]:
len(documents)

10

In [10]:
from uuid import uuid4
uuid4()

UUID('93ae3732-f0e6-430f-b6bb-7d3e1fa66a34')

In [11]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)



['5a6eef95-84e3-4301-a27e-7442bef07c74',
 '74a693cf-170b-4b8f-b13c-55609f760692',
 '11f40501-2444-433b-8f68-5dd0d6c1bf35',
 '6e1eb3a6-81b8-456a-981b-fe24020b8ef3',
 '6d270bcb-25c6-4191-9562-90f6002a7363',
 '94034175-b8de-4ad5-857f-caa766f99726',
 'a1cee64c-366e-458d-88cd-fc6e91b39d9e',
 '2d8646be-bf08-4e20-be23-395483ab7f57',
 '2783bccd-d615-4a82-a55d-7d065b558637',
 'b50904b3-c426-49d5-8e89-4f6e6a6dfa4e']

In [12]:
# Data Retrive

results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [13]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?",
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.667716] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.667716] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.577374] I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]
* [SIM=0.577374] I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]


In [14]:
from re import search
def answer_to_user(query :str):
  # vector search
  results = vector_store.similarity_search(query, k=2)

  # pass to Model vector result + user query

  final_answer = model(result, query)

  return final_answer